In [ ]:
# %load_ext autoreload
# %autoreload 2
import logging

import numpy as np
import pandas as pd
from bballref import data_pulls as dp
from bballref import general_utility as gu
from bballref.config import config

logger = logging.getLogger(__name__)

In [ ]:
standardpitchingdata = dp.standardpitchingdata(config.year)

In [ ]:
logger.info("Fixed infinity ERA's to a string of 10000")
standardpitchingdata.loc[standardpitchingdata.ERA == "inf", "ERA"] = "10000"
logger.info("Assigning columns to int/float if they apply")
gu.assignmetadata(standardpitchingdata, "STANDARD_PITCHING_DATA")
gu.write_parquet(
    standardpitchingdata, config.data_path + "\\standardpitchingdata.parquet.gzip"
)

In [ ]:
playerarray = np.unique(standardpitchingdata["Player_CDE"].to_numpy())
firstitem = playerarray[0]

In [ ]:
counter = 0
for player_cde in playerarray:
    data = dp.pitchingplatoonsplits(player_cde, config.year)
    if player_cde == firstitem:
        pitchingplatoonsplits = data
    else:
        pitchingplatoonsplits = pd.concat([pitchingplatoonsplits, data])
    if counter % 10 == 0:
        logger.info("Ticker Update: " + player_cde)
    counter += 1

In [ ]:
gu.write_parquet(
    pitchingplatoonsplits, config.data_path + "\\pitchingplatoonsplits.parquet.gzip"
)